<a href="https://colab.research.google.com/github/dhnanjay/HuggingFace/blob/main/Together_API_Langchain_LLaMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.7 MB/s eta 0:00:00


## LLaMA 2 on Together API

In [ ]:
import os

os.environ["TOGETHER_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.263
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


In [ ]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [ ]:
# print the first model's name
print(models[3]['name']), print(models[52]['name'])

EleutherAI/pythia-1b-v0
togethercomputer/llama-2-13b


(None, None)

In [ ]:
for idx, model in enumerate(models):
    print(idx, model['name'])

0 EleutherAI/gpt-j-6b
1 EleutherAI/gpt-neox-20b
2 EleutherAI/pythia-12b-v0
3 EleutherAI/pythia-1b-v0
4 EleutherAI/pythia-2.8b-v0
5 EleutherAI/pythia-6.9b
6 HuggingFaceH4/starchat-alpha
7 NousResearch/Nous-Hermes-13b
8 NousResearch/Nous-Hermes-Llama2-13b
9 NumbersStation/nsql-6B
10 OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5
11 OpenAssistant/stablelm-7b-sft-v7-epoch-3
12 bigcode/starcoder
13 databricks/dolly-v2-12b
14 databricks/dolly-v2-3b
15 databricks/dolly-v2-7b
16 google/flan-t5-xl
17 huggyllama/llama-13b
18 huggyllama/llama-30b
19 huggyllama/llama-65b
20 huggyllama/llama-7b
21 lmsys/fastchat-t5-3b-v1.0
22 lmsys/vicuna-13b-v1.3
23 lmsys/vicuna-7b-v1.3
24 prompthero/openjourney
25 runwayml/stable-diffusion-v1-5
26 stabilityai/stable-diffusion-2-1
27 stabilityai/stable-diffusion-xl-base-1.0
28 stabilityai/stablelm-base-alpha-3b
29 stabilityai/stablelm-base-alpha-7b
30 tatsu-lab/alpaca-7b-wdiff
31 timdettmers/guanaco-7b
32 togethercomputer/GPT-JT-6B-v1
33 togethercomputer/GPT-JT-Mo

In [ ]:
print(models[55]['name'])

togethercomputer/llama-2-7b-chat


In [ ]:
together.Models.start("togethercomputer/llama-2-70b-chat")

{'success': True,
 'value': '026632c004680aa0d545c2f310def6925da42cbe696a564a70b28bd9e5918ab9'}

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:
test_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=512
)

In [ ]:
type(test_llm), test_llm.model, test_llm.temperature

(__main__.TogetherLLM, 'togethercomputer/llama-2-70b-chat', 0.1)

In [ ]:
test_llm("What are the olympics? ")

"The Olympics are a major international sporting event that takes place every four years. The Olympic Games are considered the world's foremost sports competition. More than 200 countries and 10,000 athletes participate in the Games, which are held over a period of several weeks. The Olympic Games are a symbol of the world's diverse cultures coming together in a celebration of athletic achievement.\n\nThe Olympic Games have a rich history. The first recorded Olympic Games took place in 776 B.C. in ancient Greece. The Games were held every four years for over 1,000 years until they were abolished in 393 A.D. by the Roman Emperor Theodosius. In 1894, the French educator Pierre de Coubertin founded the International Olympic Committee (IOC), which led to the revival of the Olympic Games. The first modern Olympic Games were held in 1896 in Athens, Greece.\n\nThe Olympic Games are divided into two main categories: summer and winter games. The Summer Olympics feature sports such as track and 

## LangChain

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
from langchain import PromptTemplate,  LLMChain

In [ ]:
# llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=512
)

In [ ]:
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to French:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to French:

 {text}[/INST]


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
text = "how are you today?"
output = llm_chain.run(text)

print(output)

 Sure, here's the translation:

Comment allez-vous aujourd'hui?


### Summarization

In [ ]:
instruction = "Summarize the following article for me {text}"
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an expert and summarization and expressing key ideas succintly
<</SYS>>

Summarize the following article for me {text}[/INST]


In [ ]:
def count_words(input_string):
    words = input_string.split(" ")
    return len(words)

text = '''Twitter (now X) CEO Linda Yaccarino claims usage at ‘all time high’ in memo to staff
Twitter’s (now X’s) newly established CEO Linda Yaccarino touts the company’s success and X’s future plans in a company-wide memo obtained by CNBC. The exec once again claims, without sharing any specific metrics, that the service’s usage is at an “all time high,” and hints at what’s to come in terms of new product experiences for the newly rebranded platform.

The service formerly known as Twitter has been working to become more than just a social network and more of an “everything app,” as owner Elon Musk dubbed it.

As the Telsa and Space X exec explained in October 2022, telegraphing Twitter’s eventual rebranding, buying Twitter was meant to be “an accelerant to creating X, the everything app.”


His grand plan has been to create an app that allows creators to monetize their content, then later moves into payments services and even banking, Musk remarked during a Twitter Spaces livestream with advertisers in November. At the time, he even mentioned the possibility of establishing money market accounts on Twitter that would pay a high-interest rate to attract consumers to X.

Those possible product concepts were again referenced in Yaccarino’s new missive, when she writes, “Our usage is at an all time high and we’ll continue to delight our entire community with new experiences in audio, video, messaging, payments, banking – creating a global marketplace for ideas, goods, services, and opportunities.”

Twitter, now X, has already implemented some of Musk’s ideas around videos and creator monetization. In May, the company began allowing subscribers to upload two-hour videos to its service, which advertiser Apple then leveraged when it released the entire first episode of its hit Apple TV+ show “Silo” on the platform. Fired Fox News host Tucker Carlson had been posting lengthy videos to Twitter as well, until ordered to stop by the network.

In addition, earlier this month, Twitter began sharing ad revenue with verified creators.

However, all is not well at Twitter X, whose traffic — at least by third-party measurements — has been dropping. Data from web analytics firm Similarweb indicated Twitter’s web traffic declined 5% for the first two days its latest rival, Instagram Threads, became generally available, compared with the week prior. Plus, Similarweb said Twitter’s web traffic was down 11% compared with the same days in 2022. Additionally, Cloudflare CEO Matthew Prince earlier this month tweeted a graph of traffic to the Twitter.com domain that showed “Twitter traffic tanking,” he said.


Yaccarino subtly pushed back at those reports at the time, claiming that Twitter had its largest usage day since February in early July. She did not share any specific metrics or data. At the same time, however, the company was quietly blocking links to Threads.net in Twitter searches, suggesting it was concerned about the new competition.

Today, Yaccarino repeats her vague claims around X’s high usage in her company-wide memo even as analysts at Forrester are predicting X will either shut down or be acquired within the next 12 months and numerous critics concur that the X rebrand is destined to fail.

Yaccarino’s memo, otherwise, was mostly a lot of cheerleading, applauding X’s team for their work and touting X’s ability to “impress the world all over again,” as Twitter once did.

The full memo, courtesy of CBNC, is below:

Hi team,

What a momentous weekend. As I said yesterday, it’s extremely rare, whether it’s in life or in business, that you have the opportunity to make another big impression. That’s what we’re experiencing together, in real time. Take a moment to put it all into perspective.

17 years ago, Twitter made a lasting imprint on the world. The platform changed the speed at which people accessed information. It created a new dynamic for how people communicated, debated, and responded to things happening in the world. Twitter introduced a new way for people, public figures, and brands to build long lasting relationships. In one way or another, everyone here is a driving force in that change. But equally all our users and partners constantly challenged us to dream bigger, to innovate faster, and to fulfill our great potential.

With X we will go even further to transform the global town square — and impress the world all over again.

Our company uniquely has the drive to make this possible. Many companies say they want to move fast — but we enjoy moving at the speed of light, and when we do, that’s X. At our core, we have an inventor mindset — constantly learning, testing out new approaches, changing to get it right and ultimately succeeding.

With X, we serve our entire community of users and customers by working tirelessly to preserve free expression and choice, create limitless interactivity, and create a marketplace that enables the economic success of all its participants.

The best news is we’re well underway. Everyone should be proud of the pace of innovation over the last nine months — from long form content, to creator monetization, and tremendous advancements in brand safety protections. Our usage is at an all time high and we’ll continue to delight our entire community with new experiences in audio, video, messaging, payments, banking – creating a global marketplace for ideas, goods, services, and opportunities.

Please don’t take this moment for granted. You’re writing history, and there’s no limit to our transformation. And everyone, is invited to build X with us.

Elon and I will be working across every team and partner to bring X to the world. That includes keeping our entire community up to date, ensuring that we all have the information we need to move forward.

Now, let’s go make that next big impression on the world, together.

Linda'''

count_words(text)

940

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

98
 Linda Yaccarino, CEO of Twitter (now rebranded as X), claims that the platform's usage is at an
"all-time high" in a company-wide memo, despite reports of declining traffic. She touts the
company's success and future plans, including new product experiences in audio, video, messaging,
payments, and banking. The memo follows Elon Musk's vision for X as an "everything app" and his
plans to create a global marketplace for ideas, goods, and services. However, some analysts predict
that X will shut down or be acquired within the next 12 months, and critics question the success of
the rebrand.




## Simple Chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [ ]:
instruction = "Chat History:\n\n{chat_history} \n\nHuman: {user_input}\n\n Assistant:"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

Human: {user_input}

 Assistant:[/INST]


In [ ]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
llm_chain.predict(user_input="Hi, my name is Sam")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

Human: Hi, my name is Sam

 Assistant:[/INST]

> Finished chain.


" Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?"

In [ ]:
llm_chain.predict(user_input="Can you tell me about yourself.")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit? 

Human: Can you tell me about yourself.

 Assistant:[/INST]

> Finished chain.


" Sure! I'm just an AI, I don't have a personal life or experiences like humans do. However, I can tell you that I'm here to help answer any questions you might have, provide information, or assist with tasks. I'm designed to be a helpful tool for people, and I'm always happy to help in any way I can. Is there something specific you'd like to know or discuss?"

In [ ]:
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?
Human: Can you tell me about yourself.
AI:  Sure! I'm just an AI, I don't have a personal life or experiences like humans do. However, I can tell you that I'm here to help answer any questions you might have, provide information, or assist with tasks. I'm designed to be a helpful tool for people, and I'm always happy to help in any way I can. Is there something specific you'd like to know or discuss? 

Human: Today is Friday. What number day of the week is that?

 Assistant:[/INST]

> Finished chain.


' Sure, I can help you with that! Today is indeed Friday, which is the fifth day of the week. The number of the day is 5.'

In [ ]:
llm_chain.predict(user_input="what is the day today?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?
Human: Can you tell me about yourself.
AI:  Sure! I'm just an AI, I don't have a personal life or experiences like humans do. However, I can tell you that I'm here to help answer any questions you might have, provide information, or assist with tasks. I'm designed to be a helpful tool for people, and I'm always happy to help in any way I can. Is there something specific you'd like to know or discuss?
Human: Today is Friday. What number day of the week is that?
AI:  Sure, I can help you with that! Today is indeed Friday, which is the fifth day of the week. The number of the day is 5. 

Human: what is the day today?

 Assistant:[/IN

' Sure, I can help you with that! Today is Friday.'

In [ ]:
llm_chain.predict(user_input="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?
Human: Can you tell me about yourself.
AI:  Sure! I'm just an AI, I don't have a personal life or experiences like humans do. However, I can tell you that I'm here to help answer any questions you might have, provide information, or assist with tasks. I'm designed to be a helpful tool for people, and I'm always happy to help in any way I can. Is there something specific you'd like to know or discuss?
Human: Today is Friday. What number day of the week is that?
AI:  Sure, I can help you with that! Today is indeed Friday, which is the fifth day of the week. The number of the day is 5.
Human: what is the day today?
AI:  Sure, I can h

' Sure, I can help you with that! Your name is Sam.'

In [ ]:
llm_chain.predict(user_input="Can you tell me about the olympics")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?
Human: Can you tell me about yourself.
AI:  Sure! I'm just an AI, I don't have a personal life or experiences like humans do. However, I can tell you that I'm here to help answer any questions you might have, provide information, or assist with tasks. I'm designed to be a helpful tool for people, and I'm always happy to help in any way I can. Is there something specific you'd like to know or discuss?
Human: Today is Friday. What number day of the week is that?
AI:  Sure, I can help you with that! Today is indeed Friday, which is the fifth day of the week. The number of the day is 5.
Human: what is the day today?
AI:  Sure, I can h

" Sure, I'd be happy to help! The Olympics are a major international sporting event that takes place every four years, featuring a wide variety of sports and athletes from around the world. The Olympic Games have a rich history, dating back to ancient Greece, and have evolved over time to include new sports and events. The Summer Olympics and Winter Olympics are held alternately, with the Summer Olympics featuring sports such as track and field, swimming, and gymnastics, while the Winter Olympics feature sports such as skiing, skating, and ice hockey.\n\nThe Olympics are organized by the International Olympic Committee (IOC), which is responsible for selecting the host city, determining the sports and events to be included, and enforcing the rules and regulations of the Games. The Olympics are a symbol of international unity and athletic achievement, and are watched by millions of people around the world.\n\nIs there anything specific you'd like to know about the Olympics?"

In [ ]:
llm_chain.predict(user_input="What have we talked about in this Chat?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?
Human: Can you tell me about yourself.
AI:  Sure! I'm just an AI, I don't have a personal life or experiences like humans do. However, I can tell you that I'm here to help answer any questions you might have, provide information, or assist with tasks. I'm designed to be a helpful tool for people, and I'm always happy to help in any way I can. Is there something specific you'd like to know or discuss?
Human: Today is Friday. What number day of the week is that?
AI:  Sure, I can help you with that! Today is indeed Friday, which is the fifth day of the week. The number of the day is 5.
Human: what is the day today?
AI:  Sure, I can h

" Sure, I can help you with that! In this chat, you and I have talked about:\n\n1. Your name, which is Sam.\n2. The day of the week, which is Friday.\n3. The number of the day, which is 5.\n4. The Olympics, including a brief overview of the history and organization of the Games, as well as the sports and events that are included.\n\nIs there anything else you'd like to know or discuss?"

In [ ]:
together.Models.stop("togethercomputer/llama-2-70b-chat")

{'success': True}